In [8]:
import json
import pandas as pd


## TODO
* Make `add_rna_seq_data` and `create_or_update` more efficient
* Add arabidopsis orthologues to DB
  * Get from accession numbers (NIH and )
* Connect front end to db for RNA-seq
  * search by:
    * At gene
    * [X] Xe gene
    * GO terms
  * Select experiment
* add species.id to experiments model so that i can filter by exp for specific species
* add DEGs table and DEG data
  * filter expression data by DEGs
* Fix up `gene_query_page.py`
  * move the functions to the database
* Maybe clear the textbox on expression_page when user changes something in sidebar and add text in box physically rather than placeholder
* display genes for different GO terms in different panels?
* 
## Done
* Show which queried Xe genes arent in DB
* plot expression values for Xerophyta genes
  * log or normalised
* Query expression data by experiment in DB (currently searches all of expression table)
* Fix button stopping showing raw data

## Questions
- How should i handle the clusters because they aren't consistent?
- I need the humilis fasta file from nicci

## 01-01-2025 Add RNAseq data to DB
Need to add the X. elegans RNA-seq data to the db. 

Steps:
1. Add tables to `models.py`  and add info to DB schema with Alembic  
2. Format data into long data format  

### 1. Add table
**Set up alembic** 

`alembic init alembic`  
... chatGPT for other steps  

**Create a migration script**   

`alembic revision --autogenerate -m "Your migration description here"`  
fill in script
`alembic upgrade head`  


Added two tables: `Gene_expressions` and `Experiments`

### 2. Format the data

Use the `data_tidier.py` script and the below python cells



In [11]:
import data_tidier as dt
import pandas as pd

# Load the RNA expression file
file_path = "all_data/Michael_RNAseq/Xe_seedlings (updated)/Xe_seedlings DESeq2 normalised counts table.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Transform the data to long format
long_df = dt.transform_to_long(df)

long_df.to_csv("all_data/Michael_RNAseq/Xe_seedlings (updated)/Xe_seedlings_DESeq2_normalised_counts_table_tidy.csv", index=False)
print(long_df.head())




/Users/olivermarketos/Library/Mobile Documents/com~apple~CloudDocs/2024/programming_projects/app/xerophyta_data_app/data_tidier.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gene_name = row[0]  # First column is the gene name


           gene_name     species treatment replicate time  expression
0  Xele.ptg000001l.1  X. elegans        De        R2  T00   24.091222
1  Xele.ptg000001l.1  X. elegans        De        R2  T03   21.161189
2  Xele.ptg000001l.1  X. elegans        De        R2  T06   10.594839
3  Xele.ptg000001l.1  X. elegans        De        R2  T09   15.733636
4  Xele.ptg000001l.1  X. elegans        De        R2  T12   18.745990


In [12]:
df = dt.add_log2(long_df) # add log2 transformation
df = dt.format_time_points(df) # format timepoint
df.rename(columns={"expression": "normalised_expression"}, inplace=True)
print(df.head())

df.to_csv("all_data/Michael_RNAseq/Xe_seedlings (updated)/Xe_seedlings_DESeq2_normalised_counts_table_tidy_for_db.csv", index=False)

           gene_name     species treatment replicate  time  \
0  Xele.ptg000001l.1  X. elegans        De        R2     0   
1  Xele.ptg000001l.1  X. elegans        De        R2     3   
2  Xele.ptg000001l.1  X. elegans        De        R2     6   
3  Xele.ptg000001l.1  X. elegans        De        R2     9   
4  Xele.ptg000001l.1  X. elegans        De        R2    12   

   normalised_expression  log2_expression  
0              24.091222         4.649111  
1              21.161189         4.469963  
2              10.594839         3.535411  
3              15.733636         4.064679  
4              18.745990         4.303488  


### 3. Add to database
Added X. elegans seedling time course data.  



In [ ]:
import db_manager as dbm


species_id = "X. elegans"
experiment_name = "xe_seedlings_time_course"
rna_seq_data = pd.read_csv("all_data/Michael_RNAseq/Xe_seedlings (updated)/Xe_seedlings_DESeq2_normalised_counts_table_tidy_for_db.csv")

dbm.add_experiment(experiment_name, "time course of X. elegans seedlings")
dbm.add_rna_seq_data(rna_seq_data, species_id, experiment_name )


## 02.01.2025 Connect frontend to DB

In [2]:
import db_manager as dbm
import db

species_id = "X. elegans"
experiment_name = "xe_seedlings_time_course"
database = db.DB()

database.link_experiment_to_species(experiment_name, species_id)

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper[Species(species)]'. Original exception was: Could not determine join condition between parent/child tables on relationship Species.experiment - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

## 5-01-2025 Add DEG table

- added DEG tables and data for X elegans seedlings.
- can filter by DEGs now
- Can also query expression data based on GO terms

#### Testing DEG implementation
"12","Xele.ptg000001l.15","ReT48","Up-regulated","None","None"
"46","Xele.ptg000001l.50","None","None","DeT09","Down-regulated"
"137","Xele.ptg000001l.159","ReT04","Up-regulated","DeT12","Down-regulated"

Xele.ptg000001l.15
Xele.ptg000001l.50
Xele.ptg000001l.159

nice example: nuclear ubiquitin ligase complex
  - 12 genes
  - 6 degs
  - 3 up 
  - 3 down

In [11]:
import db 

database = db.DB()

gene = "Xsch.v2.MJHO01000001.1.1119"
genes =database.get_genes_by_go_term_or_description(["jasmonic acid and ethylene-dependent systemic resistance"], "X. elegans")
gene_names = [gene.gene_name for gene in genes]

print(len(gene_names))

data = database.get_gene_expression_data(gene_names, "xe_seedlings_time_course")
print(data)

79
      gene_id  normalised_expression  log2_expression treatment  time  \
0         982              78.558331         6.313941        De     0   
1         982              24.335368         4.663081        De     3   
2         982              37.081936         5.251035        De     6   
3         982              19.435668         4.353018        De     9   
4         982              13.747060         3.882355        De    12   
...       ...                    ...              ...       ...   ...   
3061    19131              57.588840         5.872554        Re     2   
3062    19131              51.534981         5.715206        Re     4   
3063    19131              61.750461         5.971554        Re     8   
3064    19131              78.532012         6.313464        Re    12   
3065    19131              99.976005         6.657869        Re    48   

     replicate            gene_name  
0           R2  Xele.ptg000001l.148  
1           R2  Xele.ptg000001l.148  
2     

In [21]:
import models
from sqlalchemy import func

def get_go_terms_with_fewest_genes(threshold=0):
    """
    Query GO terms linked to the fewest genes in ascending order of gene count.

    Parameters:
        session: SQLAlchemy session object.

    Returns:
        List of tuples containing GO term ID, GO term name, and gene count.
    """
    database = db.DB()
    session = database.session
    query = (
        session.query(
            models.GO.go_id,          # Select the GO ID
            models.GO.go_name,        # Select the GO Name
            func.count(models.Gene.id).label("gene_count")  # Count distinct genes
        )
        .join(models.annotations_go, models.annotations_go.c.go_id == models.GO.id)  # Join GO to annotations_go
        .join(models.Annotation, models.Annotation.id == models.annotations_go.c.annotation_id)  # Join annotations_go to annotations
        .join(models.Gene, models.Gene.id == models.Annotation.gene_id)  # Join annotations to genes
        .join(models.Species, models.Species.id == models.Gene.species_id)  # Join genes to species
        .group_by(models.GO.go_id, models.GO.go_name)  # Group by GO ID and name
        .having(func.count(models.Gene.id) > threshold)  # Filter by gene count greater than 0
        .order_by(func.count(models.Gene.id).asc())  # Order by gene count in ascending order
        .filter(models.Species.name == "X. elegans") 
    )

    return query.all()

print(get_go_terms_with_fewest_genes(13))

[('C:GO:0000123', 'C:histone acetyltransferase complex', 14), ('C:GO:0000152', 'C:nuclear ubiquitin ligase complex', 14), ('C:GO:0005662', 'C:DNA replication factor A complex', 14), ('C:GO:0005784', 'C:Sec61 translocon complex', 14), ('C:GO:0009842', 'C:cyanelle', 14), ('C:GO:0030137', 'C:COPI-coated vesicle', 14), ('C:GO:0030863', 'C:cortical cytoskeleton', 14), ('C:GO:0031467', 'C:Cul7-RING ubiquitin ligase complex', 14), ('C:GO:0035061', 'C:interchromatin granule', 14), ('C:GO:0042651', 'C:thylakoid membrane', 14), ('C:GO:0043073', 'C:germ cell nucleus', 14), ('C:GO:0043209', 'C:myelin sheath', 14), ('C:GO:0045273', 'C:respiratory chain complex II (succinate dehydrogenase)', 14), ('C:GO:0046695', 'C:SLIK (SAGA-like) complex', 14), ('C:GO:0070505', 'C:pollen coat', 14), ('C:GO:0097381', 'C:photoreceptor disc membrane', 14), ('C:GO:0097550', 'C:transcription preinitiation complex', 14), ('C:GO:0098835', 'C:presynaptic endocytic zone membrane', 14), ('C:GO:1990062', 'C:RPAP3/R2TP/prefo